In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import torch
from torch import nn
from torch.utils.data import TensorDataset, random_split, DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')
%cd /content/MyDrive/MyDrive

Mounted at /content/MyDrive


In [ ]:
base_dir = 'yelp_dataset'

/content/MyDrive/MyDrive


In [ ]:
df_b = pd.read_csv(f'{base_dir}/business.csv')

In [ ]:
df_b['id'] = df_b.index

In [ ]:
df_b = df_b.drop(['address','city','state','postal_code','latitude','longitude','is_open','hours'],axis=1)

In [ ]:
bid_to_id = {}
for i,r in df_b.iterrows():
    bid_to_id[r['business_id']] = r['id']

In [ ]:
len(df_b)

7314

In [ ]:
df_users = pd.read_csv(f'{base_dir}/users.csv')

In [ ]:
df_users['id'] = df_users.index

In [ ]:
uid_to_id = {}
for i,r in df_users.iterrows():
    uid_to_id[r['user_id']] = r['id']

In [ ]:
df_users = df_users.drop(['yelping_since','elite','friends'],axis=1)

In [ ]:
len(df_users)

68587

In [ ]:
def one_hot(idx, length):
   a = torch.zeros(length)
   a[idx] = 1
   return a

In [ ]:
one_hot(1,10)

tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
df_reviews = pd.read_csv(f'{base_dir}/reviews.csv')

In [ ]:
df_reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,score,negative,neutral,positive,compound
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,"{'neg': 0.0, 'neu': 0.893, 'pos': 0.107, 'comp...",0.000,0.893,0.107,0.8597
1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,"{'neg': 0.0, 'neu': 0.66, 'pos': 0.34, 'compou...",0.000,0.660,0.340,0.9588
2,Xs8Z8lmKkosqW5mw_sVAoA,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5.0,0,0,0,My absolute favorite cafe in the city. Their b...,2014-11-12 15:30:27,"{'neg': 0.025, 'neu': 0.738, 'pos': 0.237, 'co...",0.025,0.738,0.237,0.9679
3,G_5UczbCBJriUAbxz3J7Tw,clWLI5OZP2ad25ugMVI8gg,x4XdNhp0Xn8lOivzc77J-g,5.0,0,0,0,Best thai food in the area. Everything was au...,2013-08-15 15:27:51,"{'neg': 0.0, 'neu': 0.586, 'pos': 0.414, 'comp...",0.000,0.586,0.414,0.8910
4,DyrAIuKl60j_X8Yrrv-kpg,mNsVyC9tQVYtzLOCbh2Piw,MWmXGQ98KbRo3vsS5nZhMA,5.0,1,0,0,I recently had dinner here with my wife over t...,2014-10-27 02:47:28,"{'neg': 0.026, 'neu': 0.753, 'pos': 0.221, 'co...",0.026,0.753,0.221,0.9646


In [ ]:
df_reviews['user_id'] = df_reviews['user_id'].map(uid_to_id)
df_reviews['business_id'] = df_reviews['business_id'].map(bid_to_id)

In [ ]:
df_ncf = df_reviews.drop(['review_id','useful','funny','cool','text','date','score','negative','neutral','positive'],axis=1)

In [ ]:
df_ncf['stars'].min()

1.0

In [ ]:
len(df_ncf)

626103

In [ ]:
num_users, num_businesses = len(df_users), len(df_b)

In [ ]:
df_ncf2 = df_ncf.copy()

In [ ]:
df_ncf2['score'] = (df_ncf2['stars'] + ((df_ncf2['compound'] + 1) * 2.5)) / 10
#df_ncf2['score'] = df_ncf2['stars'] / 5

In [ ]:
df_ncf2 = df_ncf2.drop(['stars','compound'],axis=1)

In [ ]:
df_ncf2 = df_ncf2.sample(frac=1)

In [ ]:
#MATRIX FACRORIZATION
from collections import defaultdict

R_ui,R_iu = defaultdict(dict),defaultdict(dict)
for i,r in df_ncf2.iterrows():
  R_ui[int(r['user_id'])][int(r['business_id'])] = r['score']
  R_iu[int(r['business_id'])][int(r['user_id'])] = r['score']

#credit to hw6
def gradient_descent_update(U,V,K):

  mu = 0.001
  lambda_value = 0.001
  for user in U.keys():
    updates = np.zeros(K)
    for item in R_ui[user].keys():
      pred = np.inner(U[user],V[item])
      error = R_ui[user][item] - pred
      updates += error*V[item]
    final_updates = 2*mu*updates - 2*lambda_value*U[user]
    U[user] += final_updates

  for item in V.keys():
    updates = np.zeros(K)
    for user in R_iu[item].keys():
      pred = np.inner(U[user],V[item])
      error = R_iu[item][user] - pred
      updates += error*U[user]
    final_updates = 2*mu*updates - 2*lambda_value*V[item]
    V[item] += final_updates
  return U,V

K=32
epochs = 50

U,V = defaultdict(np.array),defaultdict(np.array)
maximum_val = np.sqrt(df_ncf2['score'].mean() / K)

for i,r in df_ncf2.iterrows():
  user_u = int(r['user_id'])
  item_i = int(r['business_id'])
  if user_u not in U:
    U[user_u] = np.random.random(K) * maximum_val
  if item_i not in V:
    V[item_i] = np.random.random(K) * maximum_val
    

for epoch in range(epochs):
  U, V = gradient_descent_update(U, V, K)

RMSE = 0
for i,r in df_ncf2.iterrows():
  user_u = int(r['user_id'])
  item_i = int(r['business_id'])
  RMSE += (r['score'] - (U[user_u].T @ V[item_i]))**2
RMSE /= len(df_ncf2)
RMSE = np.sqrt(RMSE)
RMSE

In [ ]:
class NCF(nn.Module):
    def __init__(self, user_input_size, item_input_size, embed_size, hidden_size, device):
        super().__init__()

        
        self.user_input_size = user_input_size
        self.item_input_size = item_input_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.device = device

        #Embedding Layers
        self.user_embed = nn.Embedding(self.user_input_size, self.embed_size)
        self.item_embed = nn.Embedding(self.item_input_size, self.embed_size)

        #Fully Connected Layers
        self.fc1 = nn.Linear(2 * self.embed_size, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.fc3 = nn.Linear(self.hidden_size, self.hidden_size)
        self.fc4 = nn.Linear(self.hidden_size, self.hidden_size)
        self.fc5 = nn.Linear(self.embed_size + self.hidden_size, 1)

        #Relu Activation
        self.relu = nn.ReLU()

        self.to(self.device)

    def forward(self, x):

        user_embed = self.user_embed(x[:,0])
        item_embed = self.item_embed(x[:,1])
        concat_embeds = torch.concat((user_embed,item_embed),axis=-1)

        
        gmf_embed = torch.tensor(np.array([U[int(x[i,0])] * V[int(x[i,1])] for i in range(x.shape[0])])).to(item_embed.dtype).to(self.device)


        out = self.fc1(concat_embeds)
        out = self.relu(out)

        out = self.fc2(out)
        out = self.relu(out)

        out = self.fc3(out)
        out = self.relu(out)
        
        out = self.fc4(out)
        out = self.relu(out)

        concat_gmf_out = torch.concat((gmf_embed, out),axis=-1)
        out = self.fc5(concat_gmf_out)
        return out


In [ ]:
def train(model, train_data, test_data, epochs, optimizer, criterion):
    
    for epoch in range(epochs):
        train_loss = 0
        test_loss = 0
        for i, (data,target) in enumerate(train_data):
          model.zero_grad()
          pred_y = model(data)
          loss = criterion(pred_y.squeeze(), target.to(torch.float32))
          train_loss += loss
          loss.backward()
          optimizer.step()

        train_loss /= len(train_data)
        print(f'Epoch {epoch+1}:\nTrain RMSE:{torch.sqrt(train_loss)}',end='')

        with torch.no_grad():
          for i, (data, target) in enumerate(test_data):
            pred_y = model(data)
            loss = criterion(pred_y.squeeze(1), target.to(torch.float32))
            test_loss += loss
        
        
        test_loss /= len(test_data)
        print(f'\tTest Loss:{torch.sqrt(test_loss)}')



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
X, y = torch.tensor(df_ncf2.iloc[:,:-1].values).to(device), torch.tensor(df_ncf2.iloc[:,-1].values).to(device)
train_data, test_data = random_split(TensorDataset(X,y), [0.8,0.2])
train_data = DataLoader(train_data, batch_size=16)
test_data = DataLoader(test_data, batch_size=8)

In [ ]:

embed_size = 32
hidden_size = 2048

model = NCF(user_input_size=num_users, item_input_size=num_businesses, \
            embed_size=embed_size, hidden_size=hidden_size, device=device)

learning_rate = 9e-4
reg_rate = 1e-7
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=reg_rate)
criterion = nn.MSELoss()
epochs = 5

train(model, train_data, test_data, epochs, optimizer, criterion)

Epoch 1:
Train RMSE:0.19745971262454987	Test Loss:0.1909676045179367
Epoch 2:
Train RMSE:0.1897418051958084	Test Loss:0.19009818136692047
Epoch 3:
Train RMSE:0.1865888237953186	Test Loss:0.1896887570619583
Epoch 4:
Train RMSE:0.183143749833107	Test Loss:0.19033466279506683
Epoch 5:
Train RMSE:0.17932623624801636	Test Loss:0.19325464963912964


In [ ]:
torch.save(model.state_dict(), 'ncf_gmf_vader_params.pth')